## QR法

本节讨论数值求解一个矩阵的所有特征值的一般方法. 同时这个方法也给出矩阵的 (数值) 舒尔分解.

### 上 Hessenberg 化

上 Hessenberg 化是指将矩阵通过相似变换变成形如如下的 上 Hessenberg 矩阵 (上三角 + 下对角线):
$$Q^*AQ = H = \left[\begin{matrix}h_{11}  &  h_{12} &  \dotsc  &  h_{1,n-1} &  h_{1n} \\
 h_{21} &  h_{22}  &  \dotsc  &  h_{2,n-1} &  h_{2n} \\
\ &  h_{32}  &  \dotsc  &  h_{3,n-1}  &  h_{3n} \\
\ &   & \ddots &\vdots & \vdots\\
\ &   &   &  h_{n,n-1}  &  h_{nn}\end{matrix}\right].$$

这个可以通过 Householder 变换完成.

先找 Householder 矩阵 $Q_1$ 使得 $Q_1^*AQ_1$ 的第一列从第三个元素起都是零. 然后递归处理右下角 $(n-1)\times (n-1)$ 的子矩阵, 以此类推. 

设 $Q_1 = I - 2vv^*$, 则 $v$ 可如下确定: $v$ 的第 $2$ 到 $(n-1)$ 个元素的部分使 $A$ 的第一列的第二个元素到最后一个元素的部分旋转成只留有第二个元素非零的向量. 而 $v$ 的第一个元素为零, 使得 $Q_1^*A$ 右乘 $Q_1$ 的时候不会再改变第一列.

In [14]:
import numpy as np
from math import sqrt
from numba import jit

@jit(nopython = True)
def squarenorm(x):
    return np.dot(x,x) # np.dot(x.conj(),x).real

@jit(nopython = True)
def cnorm(x):
    return x * x # x.real * x.real + x.imag * x.imag

@jit(nopython=True)
def HouseholderVector(A):
    '''
    Compute real Householder vector of norm 2. 
    
    If no Householder is needed, the second argument returns one.
    '''
    squarelength = squarenorm(A[1:]) 
    if squarelength == 0:
        return A, 1 # A, A, 1
    normal     = A.copy()
    rotation   = 1 if A[0]>=0 else -1
    normal[0] = A[0] + sqrt(squarelength + cnorm(A[0]))*rotation
    normal *= sqrt(2. / (squarelength + cnorm(normal[0])))
    return normal, 0

@jit(nopython=True)
def leftHouse(A,normal):
    '''
    left Householder transformation where ||``normal``|| = sqrt 2
    '''
    tmp = normal @ A
    n = tmp.shape[0]
    # 这里做原地修改而非生成 n*n 矩阵才能不损失效率 , 亦即 CGEMM
    # 为了使用 for 循环, 额外用到 jit 加速, 这样做比 numpy 生成 n*n 矩阵后再修改快很多倍
    for i in range(normal.shape[0]):
        for j in range(n):
            A[i,j] -= normal[i]*tmp[j]

@jit(nopython=True)
def rightHouse(A,normal):
    '''
    right Householder transformation where ||``normal``|| = sqrt 2
    '''
    tmp = A @ normal
    n = normal.shape[0]
    for i in range(tmp.shape[0]):
        for j in range(n):
            A[i,j] -= tmp[i]*normal[j]

In [16]:
@jit(nopython=True)
def Hessenberg(A):
    ''' 
    Convert a matrix to upper-Hessenberg
    '''
    n = A.shape[0] 
    Q = np.eye(n,dtype=A.dtype)
    for i in range(n-1):
        normal, invalid = HouseholderVector(A[i+1:,i])
        if invalid:
            continue
        # A ->  [I- (2nn*)] A   =   A - 2n(n* A)
        leftHouse(A[i+1:,i:],normal)
        A[i+2:,i] = 0
        rightHouse(A[:,i+1:],normal)
        rightHouse(Q[:,i+1:],normal)
    return (Q,A)

n = 5
A = np.random.randn(n*n).reshape((n,n)) #+ np.random.randn(n*n).reshape((n,n))*1j
Q , H = Hessenberg(A)
print(H)
print('||Q*AQ - H|| =',np.linalg.norm(Q.T.conj() @ A @ Q - H))

[[-1.30479815  1.1678512  -0.05352083  0.59589621  0.10371592]
 [-0.93925021 -1.37956968  0.22407899  0.72237325  0.68310967]
 [ 0.         -1.77086315  1.61041913 -2.2574651  -0.02905208]
 [ 0.          0.          1.37833429  1.72832441  1.75944682]
 [ 0.          0.          0.         -0.9148496  -0.61510176]]
||Q*AQ - H|| = 7.902155193287045


### Householder QR 迭代

将上面得到的 上 Hessenberg 矩阵 记为 $H_1$. 考虑如下 QR 迭代
$$H_m - \mu_mI = Q_mR_m\quad\quad H_{m+1} = R_mQ_m + \mu_mI.$$
其中左式为 $H_m - \mu_mI$ 的 QR 分解, 而 $\mu_m$ 应取 $H_m$ 的最右下角的元素. 可以推出
$$H_{m+1} = Q_m^TH_m Q_m.$$

实际上 $H_m - \mu_m I$ 进行 QR 分解仍然可以用 Householder 变换, 由于 Hessenberg 每列有很多零, 用时只有 $O(n^2)$.

### 双重 QR 迭代

对于实矩阵, 虽然有拟舒尔分解, 但是如果在上述迭代步中只取 $\mu_m$ 的最右下角的元素不一定收敛 (应取 $\mu_m$ 为复数且接近特征值). 假设 $\mu,\mu'\in\mathbb C$ 为 $H_m$ 右下角 $2\times 2$ 矩阵的特征值, 则假设连续两步分别取 $\mu_m$ 为 $\mu,\mu'$,
$$H_m - \mu I = Q_mR_m\quad\quad H_{m+\frac12} = R_mQ_m + \mu I.$$
$$H_{m+\frac12} - \mu 'I = Q_{m+\frac12}R_{m+\frac12}\quad\quad H_{m+1} = R_{m+\frac12}Q_{m+\frac12} + \mu 'I.$$
这两步可以合并成实 QR 迭代: 若设
$M=(H_m - \mu I)(H_m - \mu' I)\in \mathbb R^{n\times n}$ 有 QR 分解 $M = QR$, 则 $H_{m+1} = Q^*H_mQ$.

$M,Q,R$ 并不需要完全求出. 实际上, $M$ 的第一列与 $Q$ 的第一列平行, 如果知道了 $Q$ 的第一列, $H_{m+1}$ 可以通过 $H_m$ 进行 Householder-QR 过程完全确定. 因此只需要得出 $M$ 的第一列.

这里 $M$ 是实矩阵因为 $\mu,\mu'$ 如果不是实数的话两者必共轭. 该方法使得实矩阵的拟舒尔分解规避了复数运算.

<br>

此外, 上述步骤需要在不可约的 Hessenberg 矩阵中做, "不可约"表示下对角线没有零. 如果有零就需要分块.

 若矩阵的右下角接近对角阵 (若为实舒尔分解则为块对角阵), 则应将其及时分离. 在大多数情况下, 重复迭代约 $2n$ 步后矩阵会收敛. 极个别情况 (测度为 0) 该方法收敛很慢或不收敛.

In [17]:
@jit(nopython=True)
def ImplicitDoubleShift(A,Q,U,V):
    '''
    Perform a double shift on matrix A and update A,Q,U,V
    '''
    s = A[-1,-1] + A[-2,-2]
    t = A[-1,-1]*A[-2,-2] - A[-1,-2]*A[-2,-1]
    normal, invalid = HouseholderVector(np.array([
                A[0,0]*(A[0,0]-s) + A[0,1]*A[1,0] + t,
                A[1,0]*(A[0,0]+A[1,1]-s),
                A[1,0]*A[2,1]
                        ], dtype=A.dtype))
    if invalid:
        return
    leftHouse(A[:3,:],normal)
    rightHouse(A[:4,:3],normal)
    leftHouse(U[:3,:],normal)
    rightHouse(V[:,:3],normal)
    rightHouse(Q[:,:3],normal)

@jit(nopython=True)
def BuldgeChasing(A,Q,U,V):
    '''
    Convert the matrix back to Hessenberg and update A,Q,U,V
    '''
    n = A.shape[0]
    for i in range(1,n-1): 
        j = i + 3
        normal, invalid = HouseholderVector(A[i:j,i-1])
        if invalid:
            continue

        leftHouse(A[i:j,:],normal)
        A[i+1:j,i-1] = 0
        rightHouse(A[:j+1,i:j],normal)
        leftHouse(U[i:j,:],normal)
        rightHouse(V[:,i:j],normal)
        rightHouse(Q[:,i:j],normal)
        

@jit(nopython = True)
def Deflation(A,start,end,eps):
    '''
    Determine the eigenvalues to deflat and an irreduced Hessenberg matrix to operate on.
    '''
    for i in range(start,end):
        if A[i,i-1] != 0. and abs(A[i,i-1]) < eps * (abs(A[i,i])+abs(A[i-1,i-1])):
            A[i,i-1] = 0.
        
    for i in range(end-1,0,-1):
        if A[i,i-1] == 0.:
            if end - i <= 2:
                end = i
            else :
                start = i
                break
    else :
        start = 0
    return start , end

In [18]:
from sys import float_info 
from tqdm import tqdm 
#@jit
def SchurDecomposition(A,maxiter=-1,copy=1,verbose=-1,printepoch=0):
    '''
    Compute Schur Decomposition for a real matrix. Return Q,T such that Q*AQ = T.

    ``maxiter`` the maximum iteration steps permitted, defaults to 3n + 60

    ``copy`` overwrites on A if ``copy`` == 0, defaults to 1

    ``verbose`` verbose the progress if True, defaults to True when n >= 500. 

    ``printepoch`` show how many iterations it takes if True, defaults to False
    '''
    n , lastend = A.shape
    start , end = 0 , lastend

    if copy: A = A.copy()
    if verbose < 0: verbose = 1 if n >= 500 else 0
    if maxiter < 0: maxiter = 3*n + 60
    eps = float_info.epsilon
    Q , A = Hessenberg(A)
    i = 0

    if verbose:
        pbar =  tqdm(total = n)
        pbar.set_description('Deflation')

    for i in range(maxiter):
        if end <= 2:
            break

        ImplicitDoubleShift(A[start:end,start:end],Q[:,start:end],
                            A[start:end,end:],A[:start,start:end])
        BuldgeChasing(A[start:end,start:end],Q[:,start:end],
                            A[start:end,end:],A[:start,start:end])

        # deflation
        start , end = Deflation(A,start,end,eps)

        if verbose and i % 10 == 0:
            pbar.set_postfix(steps = i)
            pbar.update(lastend - end)
            lastend = end
            
    else :
        print('Warning: Convergence Failure')
        
    if verbose:
        pbar.update(lastend)
        pbar.close()
    
    if printepoch:
        print('Steps =',i)

    return (Q , A)

In [23]:
# 测试实舒尔分解
n = 5
A = np.random.randn(n*n).reshape((n,n)) #+ np.random.randn(n*n).reshape((n,n))*1j
Q , T = SchurDecomposition(A)
print(T.astype('float32'))
print('||Q*AQ - T|| =',np.linalg.norm(Q.T.conj() @ A @ Q - T))

[[-1.6506661   1.3012846  -1.0171021   0.60235596  0.9131024 ]
 [-0.60207874 -1.8583705   0.92233145 -0.07525934  0.40519723]
 [ 0.          0.          0.8104033  -2.196569   -0.9267282 ]
 [ 0.          0.          0.32557514  0.34146333 -0.02616627]
 [ 0.          0.          0.          0.          2.6812654 ]]
||Q*AQ - T|| = 3.5527660212551884e-15


In [33]:
# 利用 Frobenius 标准型特征值求解方程
def SolveEquation(coeffs):
    """
    Solve polynomial equations with Schur decomposition of Frobenius canonical form
    """
    coeffs = np.array(coeffs)
    coeffs = coeffs[1:] / coeffs[0]  # leading coefficient should be one 
    n = coeffs.size
    A = np.zeros((n, n)) # construct the Frobenius matrix
    A[0:1,:] = -coeffs 
    A[1:,:-1].flat[::n] = 1
    _ , A = SchurDecomposition(A) # get the Schur decomposition
    eigens = []
    # solve the eigenvalues of each diagonal block
    i = 0
    while i < n:
        if i+1 < n and abs(A[i+1,i]) > 1e-10: # is 2x2 block:
            delta = (A[i,i] - A[i+1,i+1])**2 + 4*A[i,i+1]*A[i+1,i]
            if delta >= 0:       delta = sqrt(delta)
            else:                delta = sqrt(-delta) * 1j
            eigens.append((A[i,i] + A[i+1,i+1] + delta) * .5)
            eigens.append((A[i,i] + A[i+1,i+1] - delta) * .5)
            i += 2
        else: # is 1x1 block
            eigens.append(A[i,i])
            i += 1
    eigens.sort(key = lambda x: x.real)
    return eigens 

coeffs = [2,5,-7,-4,5]
# solve 2x^4 + 5x^3 - 7x^2 - 4x + 5 = 0
# https://www.wolframalpha.com/input?i=2x%5E4+%2B+5x%5E3+-+7x%5E2+-+4x+%2B+5+%3D+0 
print(SolveEquation(coeffs))

[-3.306439825451153, -0.938945182564992, (0.8726925040080707+0.2089818033886869j), (0.8726925040080707-0.2089818033886869j)]


## 对称特征值

实对称矩阵 / Hermite 矩阵的特征值是一类重要的问题. 若已知矩阵是实对称 / Hermite 的, 那么利用这个信息可能产生更快的算法. 此外, 矩阵 $A$ 的奇异值分解可以转化为对称特征值问题 —— 求解$A^*A$ 的特征值. 


### Jacobi 对角化

矩阵 $A$ 是实对称矩阵. 选取 $A$ 模长最大的非对角元(之一) $a_{ij} = \overline {a_{ji}}$. 一对作用在第 $i,j$ 行与第 $i,j$ 列的左右 Givens 变换将 $a_{ij},a_{ji}$ 变成零. 重复该过程, 则最终 $A$ 趋近于对角阵. 

$$\left[\begin{matrix}c & s\\ -s & c\end{matrix}\right]
\left[\begin{matrix}a_{ii} & a_{ij}\\ a_{ij} & a_{jj}\end{matrix}\right]
\left[\begin{matrix}c & -s\\ s & c\end{matrix}\right] = 
\left[\begin{matrix}a_{ii}' & 0\\ 0 & a_{jj}'\end{matrix}\right]$$

比对右上角元素, 
$$(c^2 - s^2)a_{ij} + cs(a_{ii} - a_{jj}) = 0.$$

令 $\tau = \frac{a_{ii}-a_{jj}}{a_{ij}}$, $t = \frac{s}{c}$, 则 $t^2+2\tau t -1 = 0$. 

我们取定 $t$ 为绝对值较小的根, 数学上已证明这有利于提高收敛速度, 即 
$$t = \frac{{\rm sgn}(\tau)}{|\tau| + \sqrt{1+\tau ^2}}.$$

以及 $c = \frac{1}{\sqrt{1+t^2}}$, $s = ct$.



In [6]:
@jit(nopython=True)
def Givens(a,b,angle):
    cost , sint = angle
    tmp = cost * a + sint * b
    b[:] = (-sint).conjugate() * a + cost * b
    a[:] = tmp

@jit(nopython = True)
def JacobiDiagonalization(A, iter = -1, copy = 1):
    n = A.shape[0]
    if iter < 0: iter = 5 * n * n
    if copy: A = A.copy()
    Q = np.eye(n,dtype=A.dtype)
    
    for _ in range(iter):
        # 找到模长最大的 A[x][y]
        x , y = 0, 1
        for i in range(n):
            for j in range(i+1,n):
                if abs(A[i,j]) > abs(A[x,y]):
                    x , y = i , j

        if abs(A[x,y]) < (abs(A[x,x]) + abs(A[y,y])) * 2e-16: 
            # 如果最大值也很小, 则收敛
            break

        coeff = (A[x,x] - A[y,y]) * 0.5 /A[x,y]
        tant = 1./(abs(coeff) + math.sqrt(1+coeff*coeff))
        if coeff < 0: tant = -tant
        cost = 1./math.sqrt(1 + tant*tant)
        sint = cost * tant
        angle = (cost, sint)

        Givens(A[x],A[y],angle)
        Givens(Q[x],Q[y],angle)
        Givens(A[:,x],A[:,y],angle)
        
    return Q , A

n = 20
np.random.seed(1)
A = np.random.randn(n*n).reshape((n,n))
A = A.T @ A
Q , D = JacobiDiagonalization(A)
print('Orthogonality Loss  =',np.linalg.norm(Q.T @ Q - np.eye(n)))
print('Q.T @ D @ Q - A     =',np.linalg.norm(Q.T @ D @ Q - A))

# 观察一下 D 的非对角元的部分
V = D.copy()
for i in range(n):
    V[i,i] = 0
print('Non Diagonal Entries =',np.linalg.norm(V))

#print(sorted([D[i,i] for i in range(n)]))
#print(sorted(np.linalg.eigvalsh(A)))

Orthogonality Loss  = 1.0444528553297399e-14
Q.T @ D @ Q - A     = 4.837182546420556e-13
Non Diagonal Entries = 3.490912071133264e-14


注: 若 $A$ 是 Hermite (复) 矩阵则该方法失效.
$$\left[\begin{matrix}c & s\\ -\overline s &  c\end{matrix}\right]
\left[\begin{matrix}a_{ii} & a_{ij}\\ \overline {a_{ij}} & a_{jj}\end{matrix}\right]
\left[\begin{matrix}c & -\overline s\\ s &  c\end{matrix}\right] = 
\left[\begin{matrix}a_{ii}' & 0\\ 0 & a_{jj}'\end{matrix}\right]$$

比对右上角元素, 
$$c^2a_{ij} - |s|^2\overline{a_{ij}} +c(-\overline sa_{ii} + s a_{jj}) = 0.$$


显然一般情况下 $c\neq 0$, 注意 $a_{ii},a_{jj},c\in \mathbb R$, 若记 $s = |s|e^{i\phi}, t = \frac{|s|}{c}$, 则

$$t^2\overline{a_{ij}} + (e^{-i\phi}a_{ii} - e^{i\phi}a_{jj})t - a_{ij}  = 0.$$

考查上式实部与虚部,
$$\begin{aligned}(a_{ii} - a_{jj})t\cos\phi  + (t^2 - 1)\Re (a_{ij}) &=0 \\
-(a_{ii}+a_{jj})t\sin \phi -(t^2+1)\Im (a_{ij})&=0\end{aligned}$$

因此若 $a_{ii}\neq a_{jj}$
$$\begin{aligned}1&=\sin^2\phi+\cos^2\phi = \frac{\Re^2(a_{ij})}{(a_{ii}-a_{jj})^2}\frac{(t^2-1)^2}{t^2}
+\frac{\Im^2(a_{ij})}{(a_{ii}+a_{jj})^2}\frac{(t^2+1)^2}{t^2}\\
&=A\frac{(t^2-1)^2}{t^2}+B\frac{(t^2+1)^2}{t^2}= (A+B)(t^2+t^{-2})-2A+2B\geqslant 4B.
\end{aligned}$$
若 $\sqrt B = \left|\frac{\Im a_{ij}}{a_{ii}+a_{jj}}\right|>\frac12$ 则无解.

### 循环 Jacobi (Cyclic Jacobi)
上述 Jacobi 有一大效率的缺陷——每次都要花 $O(n^2)$ 的时间找到最大的非对角元, 但 Givens 变换只需要 $O(n)$ 的时间. 循环 Jacobi 方法提出可以依次选取非对角元 $(1,2),(1,3),\dotsc,(1,n),(2,3),(2,4),\dotsc,(n-1,n)$. 即每个非对角元都做一遍, 多来几轮. Wilkinson 证明了这是二次收敛的.

In [7]:
import numpy as np
import math 
from numba import jit

@jit(nopython=True)
def Givens(a,b,angle):
    cost , sint = angle
    tmp = cost * a + sint * b
    b[:] = (-sint).conjugate() * a + cost * b
    a[:] = tmp

@jit(nopython = True)
def JacobiDiagonalizationCyclic(A, iter = -1, copy = 1):
    n = A.shape[0]
    if iter < 0: iter = 5 * n * n
    if copy: A = A.copy()
    Q = np.eye(n,dtype=A.dtype)
    
    x, y = 0, 1
    for _ in range(iter):
        # 找到模长最大的 A[x][y]
        y += 1
        if y == n:
            x , y = x + 1 , x + 2
            if x == n - 1:
                x , y = 0 , 1
                
        if abs(A[x,y]) < (abs(A[x,x]) + abs(A[y,y])) * 2e-16: 
            # 如果最大值也很小, 则收敛
            break

        coeff = (A[x,x] - A[y,y]) * 0.5 /A[x,y]
        tant = 1./(abs(coeff) + math.sqrt(1+coeff*coeff))
        if coeff < 0: tant = -tant
        cost = 1./math.sqrt(1 + tant*tant)
        sint = cost * tant
        angle = (cost, sint)

        Givens(A[x],A[y],angle)
        Givens(Q[x],Q[y],angle)
        Givens(A[:,x],A[:,y],angle)
        
    return Q , A

n = 100
np.random.seed(1)
A = np.random.randn(n*n).reshape((n,n))
A = A.T @ A

# 比较一下效率
%timeit JacobiDiagonalization(A)
%timeit JacobiDiagonalizationCyclic(A)

1.06 s ± 75.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
201 ms ± 53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 双对角化 (Golub-Kahan Bidiagonalization)

欲计算 $A$ 的奇异值分解, 当然可以先计算 $A^*A$, 然后再 Hessenberg 化变成三对角矩阵， 但也可以按如下方法进行:
找酉阵 $U$, $V$, 使得 $UAV^*$ 是形如如下的双对角阵:
$$UAV^*=\left[\begin{matrix}\times &\times &  & & \\
\ & \times & \times & & \\
\ &  & \ddots & \ddots & \\
\ & & & \times& \times\\
\ & & &  & \times\end{matrix}\right]$$

那么 $VA^*AV^* = (VA^*U^*)(UAV^*)$ 自然是三对角阵.

找到 $U$ 和 $V$ 可以通过 Householder 变换实现: 先用左 Householder 把第一列的下面置零, 再用右 Householder 把第一行置零. 接下来处理第二行, 第二列 ...

另外, Chan [2] 指出, 若 $A\in \mathbb C^{m\times n}$ 且 $m>\frac{5}{3}n$, 可以先用 QR 分解把下面 $(m-n)\times n$ 的部分变成零, 再对上面的 $n\times n$ 部分做双对角化, 这样可以减少运算量.

双对角化完毕后并不一定要乘出三对角再运用 QR 算法求奇异值分解, 见 [1] pp. 489.

## References

1. G. Golub and C. Van Loan, Matrix Computations, The Johns Hopkins University Press, Baltimore, Maryland, 4th ed., 2013.
2. Chan, Tony F. , 
    An Improved Algorithm for Computing the Singular Value Decomposition.
    , Acm Transactions on Mathematical Software, **8.1(1982)**, 72--83.